In [3]:
'''
This should be the 5th to be runned.
This file gets the sentiment by using the german version of Bert.
The first function doesnt use parallel computing the second uses paralell computing, runt time apprx. 2h
'''
import pandas as pd
import concurrent.futures
import multiprocessing
from germansentiment import SentimentModel
from tqdm import tqdm


In [4]:
df = pd.read_csv("data/raw/comments_scores.csv")
df['text'] = df['text'].astype(str)
df_videos = pd.read_csv('videos.csv')

In [5]:
model = SentimentModel()

In [6]:
'''
For comments
'''
def analyze_sentiment(row, model):
    classes, probabilities = model.predict_sentiment([row["text"]], output_probabilities=True)

    row["sentiment"] = classes[0]
    row["prob_pos"] = probabilities[0][0][1]
    row["prob_neg"] = probabilities[0][1][1]
    row["prob_neu"] = probabilities[0][2][1]

    return row

# Assuming 'df' and 'model' are defined
total_entries = len(df)

with tqdm(total=total_entries, ncols=80) as pbar:
    def apply_analyze_sentiment(row):
        analyzed_row = analyze_sentiment(row, model)
        pbar.update(1)
        return analyzed_row

    df = df.apply(apply_analyze_sentiment, axis=1)

  0%|                                     | 46/149195 [00:04<3:47:36, 10.92it/s]


KeyboardInterrupt: 

In [ ]:
'''
For videos
'''

total_entries = len(df_videos)
with tqdm(total=total_entries, ncols=80) as pbar:
    def analyze_sentiment(row):
        global pbar

        classes, probabilities = model.predict_sentiment([row["transcript"]], output_probabilities=True)

        row["sentiment_transcript"] = classes[0]
        row["prob_pos_transcript"] = probabilities[0][0][1]
        row["prob_neg_transcript"] = probabilities[0][1][1]
        row["prob_neu_transcript"] = probabilities[0][2][1]

        pbar.update(1)  # Update the progress bar
        return row

    df_videos = df_videos.apply(analyze_sentiment, axis=1)

In [ ]:

df["sentiment"] = ""
df["prob_pos"] = 0
df["prob_neg"] = 0
df["prob_neu"] = 0


def analyze_sentiment(row, model):
    classes, probabilities = model.predict_sentiment([row["text"]], output_probabilities=True)

    row["sentiment"] = classes[0]
    row["prob_pos"] = probabilities[0][0][1]
    row["prob_neg"] = probabilities[0][1][1]
    row["prob_neu"] = probabilities[0][2][1]

    return row

def apply_analyze_sentiment(args):
    index, row, model = args
    analyzed_row = analyze_sentiment(row, model)
    return index, analyzed_row

# Assuming 'df' and 'model' are defined
total_entries = len(df)

matches1 = {}  # Dictionary to store results
args_list = [(index, row, model) for index, row in df.iterrows()]

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures1 = {executor.submit(apply_analyze_sentiment, arg): arg for arg in args_list}

    with tqdm(total=total_entries, ncols=80) as pbar:
        for future in concurrent.futures.as_completed(futures1):
            index, analyzed_row = future.result()
            df.loc[index] = analyzed_row
            pbar.update(1)



  0%|                                    | 233/149195 [00:22<3:55:27, 10.54it/s]


In [6]:
df.to_csv("data/comments_final.csv", index = False)